# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'local', 'met', 'met_2', 'pet_market', 'uk_foods']


In [11]:
# assign the uk_food database to a variable name
uk_foods_db = mongo['uk_foods']

In [8]:
# review the collections in our new database
print(uk_foods_db.list_collection_names())

['establishments']


In [10]:
# review a document in the establishments collection
pprint(uk_foods_db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f1cfa8285513e4a3677653'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [15]:
# assign the collection to a variable
establishments = uk_foods_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [16]:
# Create a dictionary for the new restaurant data
new_restaurant = {'name': "Penang Flavours",
    'BusinessType': 'Restaurant/Cafe/Canteen',
    'Address': '123 Halal Lane',
    'City': 'Greenwich',
    'Postcode': 'FT1 2AB',
    'Phone': '01234 567890',
    'RatingValue': 5,
    'latitude': '51.5074',
    'longitude': '-0.1278',
    'LocalAuthorityName': 'Foodtown Local Authority'
    }

In [17]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('66f1d3247ad850afaf90e04e'), acknowledged=True)

In [19]:
# Check that the new restaurant was inserted
halal = establishments.find_one({'name': 'Penang Flavours'})
pprint(halal)

{'Address': '123 Halal Lane',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'City': 'Greenwich',
 'LocalAuthorityName': 'Foodtown Local Authority',
 'Phone': '01234 567890',
 'Postcode': 'FT1 2AB',
 'RatingValue': 5,
 '_id': ObjectId('66f1d3247ad850afaf90e04e'),
 'latitude': '51.5074',
 'longitude': '-0.1278',
 'name': 'Penang Flavours'}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [23]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {'BusinessType': 'Restaurant/Cafe/Canteen'},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
pprint(business_type)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [25]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = 1

# update the restaurant with correct id
result = establishments.update_one(
    {'name': 'Penang Flavours'},
    {'$set': {'BusinessTypeID': business_type_id}}
)

# checking if update was successful
if result.modified_count > 0:
    print('Success')
else: 
    print('no dice')


Success


In [26]:
# Confirm that the new restaurant was updated
halal = establishments.find_one({'name': 'Penang Flavours'})
pprint(halal)

{'Address': '123 Halal Lane',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'City': 'Greenwich',
 'LocalAuthorityName': 'Foodtown Local Authority',
 'Phone': '01234 567890',
 'Postcode': 'FT1 2AB',
 'RatingValue': 5,
 '_id': ObjectId('66f1d3247ad850afaf90e04e'),
 'latitude': '51.5074',
 'longitude': '-0.1278',
 'name': 'Penang Flavours'}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [32]:
# Find how many documents have LocalAuthorityName as "Dover"
dover = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(dover)

0


In [33]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Documents deleted: {result.deleted_count}")

Documents deleted: 0


In [34]:
# Check if any remaining documents include Dover
dover = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(dover)

0


In [35]:
# Check that other documents remain with 'find_one'
pprint(uk_foods_db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f1cfa8285513e4a3677938'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [36]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [
        {'$set': {
            'geocode.longitude': {'$toDecimal': '$geocode.longitude'},
            'geocode.latitude':{'$toDecimal': '$geocode.latitude'}
        }}
    ]
)


UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [37]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [38]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {},
    [
        {
            '$set': {
                'RatingValue': {'#toInt': '$RatingValue'}
            }
        }
    ]
)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [39]:
# retiving sample docuements
sample_docuements = establishments.find().limit(10)
# Check that the coordinates and rating value are now numbers
for doc in sample_docuements:
    latitude = doc.get('geocode', {}).get('latitude')
    longitude = doc.get('geocode', {}).get('longitude')
    rating_value = doc.get('RatingValue')

    print(f"Latitude: {latitude} (Type: {type(latitude)})")
    print(f"Longitude: {longitude} (Type: {type(longitude)})")
    print(f"RatingValue: {rating_value} (Type: {type(rating_value)})")
    print("-----")


Latitude: 51.086058 (Type: <class 'bson.decimal128.Decimal128'>)
Longitude: 1.196408 (Type: <class 'bson.decimal128.Decimal128'>)
RatingValue: {'#toInt': '4'} (Type: <class 'dict'>)
-----
Latitude: 51.083812 (Type: <class 'bson.decimal128.Decimal128'>)
Longitude: 1.195625 (Type: <class 'bson.decimal128.Decimal128'>)
RatingValue: {'#toInt': '5'} (Type: <class 'dict'>)
-----
Latitude: 51.085797 (Type: <class 'bson.decimal128.Decimal128'>)
Longitude: 1.194762 (Type: <class 'bson.decimal128.Decimal128'>)
RatingValue: {'#toInt': '5'} (Type: <class 'dict'>)
-----
Latitude: 51.08084 (Type: <class 'bson.decimal128.Decimal128'>)
Longitude: 1.188537 (Type: <class 'bson.decimal128.Decimal128'>)
RatingValue: {'#toInt': '5'} (Type: <class 'dict'>)
-----
Latitude: 51.08084 (Type: <class 'bson.decimal128.Decimal128'>)
Longitude: 1.188537 (Type: <class 'bson.decimal128.Decimal128'>)
RatingValue: {'#toInt': '5'} (Type: <class 'dict'>)
-----
Latitude: 51.08084 (Type: <class 'bson.decimal128.Decimal128'>

1199
